# Silver – Tratamento e Qualidade dos Dados

## Objetivo
Neste notebook, os dados da camada **Bronze** são tratados e padronizados
para formar a camada **Silver**, garantindo melhor qualidade e consistência
para consumo analítico.

Aqui começam as transformações de negócio, preparando os dados para modelagem
analítica na camada Gold.

## O que é feito aqui
- Limpeza de dados inconsistentes ou nulos
- Padronização de nomes de colunas e tipos
- Conversões de datas e valores numéricos
- Aplicação de regras básicas de negócio
- Persistência dos dados tratados em Delta Lake

## Arquitetura
- **Fonte:** Lakehouse – Camada Bronze
- **Processo:** Transformações e validações
- **Destino:** Lakehouse – Camada Silver (Delta)


In [0]:
%sql

CREATE TABLE IF NOT EXISTS projeto_adventure_works.silver.production_product
USING DELTA
AS
SELECT
    ProductID,
    Name AS product_name,
    ProductNumber,
    Color,
    StandardCost,
    ListPrice,
    Size,
    Weight,
    SellStartDate,
    SellEndDate,
    ModifiedDate,
    _ingestion_timestamp
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY ProductID
               ORDER BY ModifiedDate DESC
           ) AS rn
    FROM projeto_adventure_works.bronze.production_product
)
WHERE rn = 1;


In [0]:


%sql


describe projeto_adventure_works.bronze.customer

In [0]:

%sql

CREATE TABLE IF NOT EXISTS projeto_adventure_works.silver.customer
USING DELTA
AS
SELECT
    CustomerID,
    PersonID,
    StoreID,
    TerritoryID,
    ModifiedDate,
    _ingetion_ts
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY CustomerID
               ORDER BY ModifiedDate DESC
           ) AS rn
    FROM projeto_adventure_works.bronze.customer
)
WHERE rn = 1;








In [0]:
%sql

CREATE TABLE IF NOT EXISTS projeto_adventure_works.silver.sales_order
USING DELTA
AS
SELECT
    SalesOrderID,
    CustomerID,
    OrderDate,
    DueDate,
    ShipDate,
    Status,
    OnlineOrderFlag,
    TotalDue,
    ModifiedDate,
    _ingestion_timestamp
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY SalesOrderID
               ORDER BY ModifiedDate DESC
           ) AS rn
    FROM projeto_adventure_works.bronze.sales_order_header
)
WHERE rn = 1;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS projeto_adventure_works.silver.sales_order_item
USING DELTA
AS
SELECT
    SalesOrderID,
    SalesOrderDetailID,
    ProductID,
    OrderQty,
    UnitPrice,
    UnitPriceDiscount,
    LineTotal,
    ModifiedDate,
    _ingestion_timestamp
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY SalesOrderDetailID
               ORDER BY ModifiedDate DESC
           ) AS rn
    FROM projeto_adventure_works.bronze.sales_order_detail
)
WHERE rn = 1;
